# Imports

In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPool2D, MaxPooling2D
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import PIL
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json

## Paths

In [2]:
TRAIN_DIR = r'images\train'
TEST_DIR = r'images\validation'

## Utils

In [3]:
def create_data_frame(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "Completed")
    
    return image_paths, labels

## Extract/Transform

In [4]:
train = pd.DataFrame()
train['image'], train['label'] = create_data_frame(TRAIN_DIR)

angry Completed
disgust Completed
fear Completed
happy Completed
neutral Completed
sad Completed
surprise Completed


In [5]:
train

,image,label
0,images\train\angry\0.jpg,angry
1,images\train\angry\1.jpg,angry
2,images\train\angry\10.jpg,angry
3,images\train\angry\10002.jpg,angry
4,images\train\angry\10016.jpg,angry
...,...,...
28816,images\train\surprise\9969.jpg,surprise
28817,images\train\surprise\9985.jpg,surprise
28818,images\train\surprise\9990.jpg,surprise
28819,images\train\surprise\9992.jpg,surprise


In [6]:
test = pd.DataFrame()
test['image'], test['label'] = create_data_frame(TEST_DIR)

angry Completed
disgust Completed
fear Completed
happy Completed
neutral Completed
sad Completed
surprise Completed


In [7]:
test

,image,label
0,images\validation\angry\10052.jpg,angry
1,images\validation\angry\10065.jpg,angry
2,images\validation\angry\10079.jpg,angry
3,images\validation\angry\10095.jpg,angry
4,images\validation\angry\10121.jpg,angry
...,...,...
7061,images\validation\surprise\9806.jpg,surprise
7062,images\validation\surprise\9830.jpg,surprise
7063,images\validation\surprise\9853.jpg,surprise
7064,images\validation\surprise\9878.jpg,surprise


In [8]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode = "grayscale")
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48, 48, 1)

    return features

## Split

In [9]:
train_features = extract_features(train['image'])

100%|██████████| 28821/28821 [00:58<00:00, 491.53it/s] 


In [10]:
test_features = extract_features(test['image'])

100%|██████████| 7066/7066 [00:50<00:00, 140.92it/s]


In [11]:
X_train = train_features / 255.0
X_test = test_features / 255.0

In [12]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [13]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [14]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [15]:
y_train[0]

array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [16]:
y_test[0]

array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)

## Model 

In [17]:
# model = Sequential()
# model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48, 48, 1)))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

# model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

# model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

# model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

# model.add(Flatten())

# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.4))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.3))

# model.add(Dense(7, activation='softmax'))

In [18]:
# model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics='accuracy')

## Train

In [19]:
# model.fit(x=X_train, y=y_train, batch_size=128, epochs=100, validation_data=(X_test, y_test))

## Load

In [20]:
# model_json = model.to_json()

In [21]:
# with open('emotion_detector.json', 'w') as f:
#     f.write(model_json)

In [22]:
# model.save('emotion_detector.h5')

In [23]:
json_file = open('emotion_detector.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)

model.load_weights('emotion_detector.h5')

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 128)       1280      
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 128)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 23, 23, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 256)       0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 256)       0

## Predict

In [25]:
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [26]:
def ef(image):
    img = load_img(image, color_mode='grayscale')
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

In [27]:
image = 'images/train/sad/42.jpg'
print('original image of sad')
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print('model prediction is', pred_label)

original image of sad
1/1 [==============================] - 0s 167ms/step
model prediction is sad
